In [1]:
from file_extraction import *
from to_dataframe import *
from to_tables import *
import pandas as pd

In [25]:
s3_clien = boto3.client('s3')
files_dic = {'academy_csv': [], 'json': [], 'txt': [], 'csv': []}

academy_csvs_file_nmes = extract_file_type(s3_clien, 'Academy', files_dic, 'csv')
df = convert_all_to_df(files_dic['academy_csv'], 'csv', academy_csvs_file_nmes)

display(df)

,name,trainer,course,date,Analytic_W1,Independent_W1,Determined_W1,Professional_W1,Studious_W1,Imaginative_W1,...,Determined_W9,Professional_W9,Studious_W9,Imaginative_W9,Analytic_W10,Independent_W10,Determined_W10,Professional_W10,Studious_W10,Imaginative_W10
0,Quintus Penella,Gregor Gomez,Business,2019-02-11,1,2,2,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Simon Murrey,Gregor Gomez,Business,2019-02-11,6,1,1,2,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gustaf Lude,Gregor Gomez,Business,2019-02-11,6,4,1,1,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Yolanda Fosse,Gregor Gomez,Business,2019-02-11,2,1,2,3,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Lynnett Swin,Gregor Gomez,Business,2019-02-11,2,2,4,5,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,Moritz Mosedall,Mohammad Velazquez,Engineering,2019-12-30,1,1,5,1,2,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Chaim Inseal,Mohammad Velazquez,Engineering,2019-12-30,1,3,3,4,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Gertruda Syddie,Mohammad Velazquez,Engineering,2019-12-30,3,1,2,8,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Thom Derwin,Mohammad Velazquez,Engineering,2019-12-30,3,7,3,3,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
files_dict, academy_csvs_file_names = extract()
dataframes = convert(files_dict, academy_csvs_file_names)

,name,trainer,course,date,Analytic_W1,Independent_W1,Determined_W1,Professional_W1,Studious_W1,Imaginative_W1,...,Determined_W9,Professional_W9,Studious_W9,Imaginative_W9,Analytic_W10,Independent_W10,Determined_W10,Professional_W10,Studious_W10,Imaginative_W10
0,Quintus Penella,Gregor Gomez,Business,2019-02-11,1,2,2,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Simon Murrey,Gregor Gomez,Business,2019-02-11,6,1,1,2,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gustaf Lude,Gregor Gomez,Business,2019-02-11,6,4,1,1,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Yolanda Fosse,Gregor Gomez,Business,2019-02-11,2,1,2,3,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Lynnett Swin,Gregor Gomez,Business,2019-02-11,2,2,4,5,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,Moritz Mosedall,Mohammad Velazquez,Engineering,2019-12-30,1,1,5,1,2,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Chaim Inseal,Mohammad Velazquez,Engineering,2019-12-30,1,3,3,4,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Gertruda Syddie,Mohammad Velazquez,Engineering,2019-12-30,3,1,2,8,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Thom Derwin,Mohammad Velazquez,Engineering,2019-12-30,3,7,3,3,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_academy_csv = dataframes['academy_csv']
display(df_academy_csv)

,name,trainer,course,date,Analytic_W1,Independent_W1,Determined_W1,Professional_W1,Studious_W1,Imaginative_W1,...,Determined_W9,Professional_W9,Studious_W9,Imaginative_W9,Analytic_W10,Independent_W10,Determined_W10,Professional_W10,Studious_W10,Imaginative_W10
0,Quintus Penella,Gregor Gomez,Business,2019-02-11,1,2,2,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Simon Murrey,Gregor Gomez,Business,2019-02-11,6,1,1,2,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gustaf Lude,Gregor Gomez,Business,2019-02-11,6,4,1,1,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Yolanda Fosse,Gregor Gomez,Business,2019-02-11,2,1,2,3,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Lynnett Swin,Gregor Gomez,Business,2019-02-11,2,2,4,5,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,Moritz Mosedall,Mohammad Velazquez,Engineering,2019-12-30,1,1,5,1,2,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Chaim Inseal,Mohammad Velazquez,Engineering,2019-12-30,1,3,3,4,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Gertruda Syddie,Mohammad Velazquez,Engineering,2019-12-30,3,1,2,8,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Thom Derwin,Mohammad Velazquez,Engineering,2019-12-30,3,7,3,3,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_csv = dataframes['csv']
display(df_csv)

,id,name,gender,dob,email,city,address,postcode,phone_number,uni,degree,invited_date,month,invited_by
0,1,Esme Trusslove,Female,04/08/1994,etrusslove0@google.es,Swindon,22056 Lerdahl Avenue,SN1,+44-295-783-0228,"Saint George's Hospital Medical School, Univer...",2:1,10.0,April 2019,Bruno Bellbrook
1,2,Matthaeus Audas,Male,NaN,maudas1@mapquest.com,Charlton,263 Nelson Trail,OX12,+44-957-728-0155,Keele University,2:1,30.0,April 2019,Doris Bellasis
2,3,Cherey Tollfree,Female,08/12/1992,ctollfree2@netvibes.com,Weston,69 Coleman Court,GU32,+44-588-749-6002,"King's College London, University of London",2:1,25.0,April 2019,Gismo Tilling
3,4,Eryn Speers,Female,NaN,espeers3@shinystat.com,Sutton,0166 Daystar Drive,CT15,+44 148 787 0613,University of Edinburgh,2:1,NaN,NaN,NaN
4,5,Theadora Berkelay,Female,03/11/1995,tberkelay4@godaddy.com,Upton,6 Mandrake Crossing,WF9,+44 (841) 468-3619,University of Leicester,2:1,2.0,April 2019,Stacey Broad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,416,Clyve Gillhespy,Male,15/12/1995,cgillhespybj@buzzfeed.com,Birmingham,33 Almo Avenue,B40,+44-904-343-2218,University of Liverpool,2:1,26.0,SEPT 2019,Bruno Bellbrook
416,417,Vaclav Pietesch,Male,09/11/1994,vpieteschbk@mac.com,Whitwell,6476 Hoffman Terrace,DL10,+44 455 631 6125,Sheffield Hallam University,2:2,12.0,SEPT 2019,Stacey Broad
417,418,Kassi Lucio,Female,24/04/1994,kluciobl@exblog.jp,Normanton,06 Fulton Center,LE15,+44-834-342-9323,University of Buckingham,2:1,3.0,SEPT 2019,Fifi Eton
418,419,Vivianna Letty,Female,NaN,vlettybm@google.com.hk,Sheffield,78314 Fisk Plaza,S1,+44-534-758-3140,Leeds Metropolitan University,1st,19.0,SEPT 2019,Bruno Bellbrook


In [8]:
df_json = dataframes['json']
display(df_json)

,name,date,tech_self_score,strengths,weaknesses,self_development,geo_flex,financial_support_self,result,course_interest
0,Stillmann Castano,22/08/2019,"{'C#': 6, 'Java': 5, 'R': 2, 'JavaScript': 2}",[Charisma],"[Distracted, Impulsive, Introverted]",Yes,Yes,Yes,Pass,Business
0,Hilary Willmore,01/08/2019,"{'Python': 1, 'C#': 4, 'Java': 2, 'C++': 4}","[Patient, Curious, Problem Solving]","[Overbearing, Chatty, Indifferent]",No,Yes,Yes,Fail,Data
0,Efrem Whipple,22/08/2019,"{'Ruby': 4, 'C++': 4}","[Courteous, Independent, Patient]","[Introverted, Impulsive, Anxious]",Yes,Yes,Yes,Pass,Business
0,Sydel Fenne,28/08/2019,"{'Java': 3, 'SPSS': 4}",[Passionate],"[Perfectionist, Sensitive]",Yes,Yes,Yes,Pass,Data
0,Michel Lebarree,07/08/2019,"{'Python': 3, 'Java': 4, 'Ruby': 1, 'R': 2, 'P...",[Versatile],"[Controlling, Perfectionist, Chatty]",Yes,Yes,Yes,Pass,Engineering
...,...,...,...,...,...,...,...,...,...,...
0,Jacky Reilingen,04/04/2019,"{'C#': 2, 'Java': 6, 'R': 1, 'SPSS': 4}",[Versatile],"[Indifferent, Intolerant, Introverted]",Yes,No,Yes,Fail,Business
0,Phillis Lyfield,10/04/2019,"{'C#': 4, 'Java': 4, 'Ruby': 4, 'PHP': 1}","[Organisation, Independent, Determined]","[Sensitive, Overbearing, Impatient]",Yes,Yes,Yes,Pass,Engineering
0,Celle Barlas,16/04/2019,"{'R': 2, 'C++': 1, 'JavaScript': 4}",[Problem Solving],[Critical],Yes,Yes,Yes,Pass,Engineering
0,Scott Duny,11/04/2019,{'Ruby': 3},"[Reliable, Perfectionism, Problem Solving]",[Stubborn],Yes,Yes,Yes,Pass,Data


In [9]:
df_txt = dataframes['txt']
display(df_txt)

,name,psychometrics_score,presentation_score,academy,date
0,Hilary Willmore,51,19,Birmingham Academy,2019-08-01
1,Orly Lorens,51,19,Birmingham Academy,2019-08-01
2,Alvie Bleackly,55,16,Birmingham Academy,2019-08-01
3,Deck Itzchaki,59,21,Birmingham Academy,2019-08-01
4,Wilt Penritt,66,25,Birmingham Academy,2019-08-01
...,...,...,...,...,...
15,Godiva Andrew,46,19,London Academy,2019-10-09
16,Doralia Gapper,55,22,London Academy,2019-10-09
17,Doe Eisold,57,20,London Academy,2019-10-09
18,Judy Finders,56,22,London Academy,2019-10-09


In [11]:
a = df_csv.loc[df_csv['name'] == 'Stillmann Castano']
b = df_csv.loc[df_csv['name'] == 'Esme Trusslove']
display(a)
display(b)

,id,name,gender,dob,email,city,address,postcode,phone_number,uni,degree,invited_date,month,invited_by
0,1,Stillmann Castano,Male,25/07/1992,scastano0@geocities.jp,Kingston,02 Wayridge Alley,DT10,+44 (980) 284-7006,Southampton Solent University,2:1,22.0,AUGUST 2019,Stacey Broad


,id,name,gender,dob,email,city,address,postcode,phone_number,uni,degree,invited_date,month,invited_by
0,1,Esme Trusslove,Female,04/08/1994,etrusslove0@google.es,Swindon,22056 Lerdahl Avenue,SN1,+44-295-783-0228,"Saint George's Hospital Medical School, Univer...",2:1,10.0,April 2019,Bruno Bellbrook


# Generate Weakness Table

In [160]:
# makes a weakness table dataframe
def generate_weakness_df(df):
    weaknesses_list = list()
    
    # for each row in the dataframe add distinct weaknesses to a list
    for index, row in df.iterrows():
        for weakness in row["weaknesses"]:
            if weakness not in weaknesses_list:
                weaknesses_list.append(weakness)
                
    # turn weakneseses list into a dataframe and add a column for the weakness id            
    weakness_table = pd.DataFrame([weaknesses_list, columns=['weakness'])
    weakness_table.index = weakness_table.index + 1
    weakness_table['weakness_id'] = weakness_table.index
    return weakness_table          

In [161]:
weakness_df = generate_weakness_df(df_json)
display(weakness_df)

,weakness,weakness_id
1,Distracted,1
2,Impulsive,2
3,Introverted,3
4,Overbearing,4
5,Chatty,5
6,Indifferent,6
7,Anxious,7
8,Perfectionist,8
9,Sensitive,9
10,Controlling,10


In [223]:
# takes talent jsons, talent csv, weakness dataframe as arguments
def generate_weakness_junc_df(df, df2, weakness_df):
    # merge talent json dataframe and talen csv dataframe and extract id and weaknesses columns
    merged_pd = pd.merge(df, df2, on='name')[['id', 'weaknesses']]
    weakness_junc_list = list()
    
    # for each row in the merged dataframe add (id, weakness) to a new dataframe for each weakness in weaknesses list
    for index, row in merged_pd.iterrows():
        applicant_id = row['id']
        weaknesses = row['weaknesses']
        
        for weakness in weaknesses:
            weakness_junc_list.append([applicant_id, weakness])
    
    weakness_junc = pd.DataFrame(weakness_junc_list, columns=['applicant_id', 'weakness'])
    
    weakness_junc = pd.merge(weakness_junc, weakness_df, on='weakness')[['applicant_id', 'weakness_id']] 
        
    return weakness_junc      

In [224]:
display(generate_weakness_junc_df(df_json, df_csv, weakness_df))

,applicant_id,weakness_id
0,1,1
1,62,1
2,73,1
3,80,1
4,88,1
...,...,...
5862,275,24
5863,279,24
5864,289,24
5865,377,24


# Generate Strengths Table

In [166]:
def generate_strengths_df(df):
    # Create a new DataFrame for strengths column only
    column_names = ['name', 'strengths']
    strengths = df[column_names]

    # Initialize an empty list to store row data
    strengths_list = []

    # Iterate over all rows
    for index, row in strengths.iterrows():
        strengths_list_row = row['strengths']

        # Iterate over each strength in the list
        for strength in strengths_list_row:
            if strength not in strengths_list:
                # Append row data to the list
                strengths_list.append(strength)

    # Create a new DataFrame from the list
    strengths_df = pd.DataFrame({'strength': strengths_list})
    strengths_df.index = strengths_df.index + 1
    strengths_df['strength_id'] = strengths_df.index
    # Return the new DataFrame
    return strengths_df

In [167]:
strengths_df = generate_strengths_df(df_json)
display(strengths_df)

,strength,strength_id
1,Charisma,1
2,Patient,2
3,Curious,3
4,Problem Solving,4
5,Courteous,5
6,Independent,6
7,Passionate,7
8,Versatile,8
9,Rational,9
10,Collaboration,10


In [175]:
# takes talent jsons, talent csv, weakness dataframe as arguments
def generate_strengths_junc_df(df, df2, strengths_df):
    merged_pd = pd.merge(df, df2, on='name')[['id', 'strengths']] 
    strengths_junc_list = list()
    
    for index, row in merged_pd.iterrows():
        applicant_id = row['id']
        strengths = row['strengths']
        
        for strength in strengths:
            strengths_junc_list.append([applicant_id, strength])
    
    strengths_junc = pd.DataFrame(strengths_junc_list, columns=['applicant_id', 'strength'])

    strengths_junc = pd.merge(strengths_junc, strengths_df, on='strength')[['applicant_id', 'strength_id']] 
        
    return strengths_junc

In [176]:
display(generate_strengths_junc_df(df_json, df2_csv, strengths_df))

,applicant_id,strength_id
0,1,1
1,40,1
2,46,1
3,81,1
4,96,1
...,...,...
5964,347,25
5965,357,25
5966,360,25
5967,361,25


# Generate Address Table

In [227]:
# makes an address table dataframe 
def generate_address_df(df):
    addresses = list()
    # for each row in the talent csv files, add each unique address, city and postcode to a list
    for index, row in df.iterrows():
                address_list = list()
                address_list.append(row["address"])
                address_list.append(row["city"])
                address_list.append(row["postcode"])
                if address_list not in addresses:
                    addresses.append(address_list)
                    
    address_table = pd.DataFrame(addresses, columns=["address", "city", "postcode"])
    address_table.index = address_table.index + 1
    return address_table

In [228]:
display(generate_address_df(df_csv))

,address,city,postcode
1,22056 Lerdahl Avenue,Swindon,SN1
2,263 Nelson Trail,Charlton,OX12
3,69 Coleman Court,Weston,GU32
4,0166 Daystar Drive,Sutton,CT15
5,6 Mandrake Crossing,Upton,WF9
...,...,...,...
4594,33 Almo Avenue,Birmingham,B40
4595,6476 Hoffman Terrace,Whitwell,DL10
4596,06 Fulton Center,Normanton,LE15
4597,78314 Fisk Plaza,Sheffield,S1


# Generate Tech Score Table - this one doesnt fully work yet

In [ ]:
def pd_to_dict(df):
    df_as_dict = df.to_dict(orient='list')
    keys = list(df_as_dict.keys())
    for key in keys:
        df_as_dict[key] = df_as_dict[key][0]
    return df_as_dict

In [ ]:
def get_tss(profiles:list):
    #Create names lists
    names = []
    technologies = []
    self_scores = []
    
    #Loop through profiles
    for profile in profiles:
        try:
            techs = list(profile['tech_self_score'].keys())
            for tech in techs:
                names.append(profile['name'].upper())
                technologies.append(tech)
                self_scores.append(profile['tech_self_score'][tech])
        except KeyError:
            continue
        
    tech_self_scores = pd.DataFrame({'name':names, 'technology':technologies, 'self_score': self_scores})
    return tech_self_scores

# Generate Candidates Table - this one doesnt fully work yet

In [30]:
def get_person_ids(applicants):
    applicants = applicants.reset_index().reset_index().drop(['index', 'id'], axis=1).rename(columns={'level_0':'person_id'})
    return applicants

 

def get_candidates(applicants, talents, sparta_day_results):
#     applicants_and_talents = pd.concat([applicants, talents], axis=1)
    applicants_and_talents = applicants.merge(talents, 
                                              left_on=['name', 'invited_date'], 
                                              right_on=['name', 'date'], 
                                              how='left')

 

#     candidates = pd.concat([sparta_day_results, applicants_and_talents], axis=1)
    candidates = applicants_and_talents.merge(sparta_day_results, 
                                              left_on=['name', 'invited_date'], 
                                              right_on=['name', 'date'], 
                                              how='left')
    
    
    return candidates

In [31]:
display(get_candidates(get_person_ids(df_csv), df_json, df_txt))

ValueError: You are trying to merge on float64 and object columns. If you wish to proceed you should use pd.concat

# Generate Academy Table

# Generate Trainers Table

# Generate Courses Table

In [50]:
def generate_courses_table(dataframe):
    courses_df  = dataframe
    courses_df = pd.DataFrame().assign(course = courses_df['course'], date = courses_df['date'])
    courses_df['course_id'] = courses_df.groupby(['course']).ngroup()
    courses_df = courses_df[['course', 'course_id', 'date']].copy()
    courses_df.set_index('course_id', inplace=True)
    courses_df.groupby(['course','date'])
    courses_df = courses_df.drop_duplicates()
    courses_df = courses_df.reset_index()
    courses_df.index = courses_df.index + 1
    courses_df['course_id'] = courses_df.index
    
    return courses_df

In [51]:
display(generate_courses_table(df_academy_csv))

,course_id,course,date
1,1,Business,2019-02-11
2,2,Business,2019-03-18
3,3,Business,2019-04-15
4,4,Business,2019-05-20
5,5,Business,2019-07-15
6,6,Business,2019-07-29
7,7,Business,2019-08-12
8,8,Business,2019-09-16
9,9,Business,2019-10-21
10,10,Business,2019-11-18


# Generate Academy Results Table